# Möbius‑Ladder Explorer (v3b) — *Sub‑0.005 φ‑scale Probe*

We zoom the φ‑scale down to **0.001 … 0.0045** in 0.0005 steps, hunting for an attractor near α.

Parameters:

* `seed_bits` = [64, 128] (higher precision)
* `phi_scale` = 0.001 – 0.0045 (Δ=0.0005)
* cycles 1‑6
* `tiny` = α⁻¹ / 10⁴

The breath‑shift map is the same as v3:

```
x ← φ_scale·φ / x
x ← x − tiny
x ← 1/x / 9
x ← 1/x
x ← x / 9
```

In [1]:
import mpmath as mp, pandas as pd
mp.mp.dps = 120
ALPHA_INV = mp.mpf('137.035999084')
ALPHA = 1/ALPHA_INV
PHI = (1 + mp.sqrt(5))/2

def fib_word_bits(n):
    a,b='0','01'
    while len(b)<n:
        a,b=b,b+a
    return b[:n]
def fib_fraction(bits):
    return mp.fsum(int(b)*mp.mpf(2)**(-(k+1)) for k,b in enumerate(fib_word_bits(bits)))


In [2]:
def breath_shift(x, tiny, phi_scale):
    x = phi_scale*PHI / x
    x = x - tiny
    x = 1/x / 9
    x = 1/x
    x = x / 9
    return x

## Narrow φ‑scale sweep

In [3]:
tiny = ALPHA_INV/1e4
rows=[]
for bits in (64,128):
    seed = fib_fraction(bits)
    for phi_scale in [0.001 + i*0.0005 for i in range(0,8)]:  # 0.001 … 0.0045
        x = seed + tiny
        for cycles in range(1,7):
            x = breath_shift(x,tiny,phi_scale)
            rel = abs((x-ALPHA)/ALPHA)
            rows.append(dict(bits=bits, phi_scale=phi_scale, cycles=cycles, rel_err=float(rel), x=mp.nstr(x,25)))
            if rel < 0.01:
                print('Hit <1%  | bits',bits,'phi_scale',phi_scale,'cycles',cycles,'rel',rel)
                raise SystemExit
df = pd.DataFrame(rows)
print('10 smallest errors:')
display(df.sort_values('rel_err').head(10))


10 smallest errors:


,bits,phi_scale,cycles,rel_err,x
36,64,0.0040,1,0.040558,0.007593315558466013624297172
84,128,0.0040,1,0.040558,0.007593315558466013622091347
78,128,0.0035,1,0.324248,0.004931201125107761919329929
30,64,0.0035,1,0.324248,0.004931201125107761921260025
42,64,0.0045,1,0.405363,0.01025542999182426763635052
90,128,0.0045,1,0.405363,0.01025542999182426763386897
72,128,0.0030,1,0.689053,0.00226908669174951021656851
24,64,0.0030,1,0.689053,0.002269086691749510218222879
18,64,0.0025,1,1.053859,-0.0003930277416087414848142677
66,128,0.0025,1,1.053859,-0.000393027741608741486192908
